<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_NER_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U --quiet spacy
!pip install "spacy-pkuseg>=0.0.27,<0.1.0"
!python -m spacy download zh_core_web_lg
# !wget https://github.com/explosion/spacy-models/releases/download/zh_core_web_md-3.1.0/zh_core_web_md-3.1.0.tar.gz
# !tar xvfz zh_core_web_md-3.1.0.tar.gz
# !python -m spacy download zh_core_web_trf

     |████████████████████████████████| 6.4 MB 4.7 MB/s 
     |████████████████████████████████| 621 kB 43.8 MB/s 
     |████████████████████████████████| 42 kB 947 kB/s 
     |████████████████████████████████| 10.1 MB 45.7 MB/s 
     |████████████████████████████████| 456 kB 44.2 MB/s 
     |████████████████████████████████| 2.4 MB 4.9 MB/s 
2021-08-10 04:11:44.022549: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 603.8 MB 8.2 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [2]:
import spacy

# spacyModel='zh_core_web_md-3.1.0/zh_core_web_md/zh_core_web_md-3.1.0'
nlp = spacy.load('zh_core_web_lg')

doc = nlp('恆指漲3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。')

for ent in doc.ents:
  print(ent.text, ent.start_char, ent.end_char, ent.label_)

3.3% 3 7 PERCENT
香港 9 11 GPE
9% 29 31 PERCENT
去年7月 34 38 DATE


In [11]:
import random
from spacy.training.example import Example

TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN'),(12,16,'PERCENT')          
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),
       (17,19,'STOCK'),(24,28,'PERCENT')       
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(3,8,'STOCK'),(11,14,'DATE'),(15,17,'DATE'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '美股異動受港股收跌拖累和黃低開低走跌14%。',
    {
        'entities':
        [
         (0,2,'FIN'),(5,7,'FIN'),(7,11,'FIN'),(11,13,'STOCK'),(13,17,'FIN'),(18,21,'PERCENT')
        ]
    }
  ),
  (
    '港隊於東京奧運創歷史佳績。',
   {
       'entities': [(3,7,'EVENT')]
   }
  )
]

ner = nlp.get_pipe('ner')

ner.add_label('STOCK')
ner.add_label('FIN')

epoch = 50

# get names of other pipes to disable them during training
pipe_exceptions = ["ner"]
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
print(f'Pipes: {other_pipes}')
with nlp.disable_pipes(*other_pipes):  # only train NER
  for itn in range(epoch):
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = spacy.util.minibatch(TRAIN_DATA, size=2)
    for batch in batches:
        texts, annotations = zip(*batch)
        example = []
        for i in range(len(texts)):
          doc = nlp.make_doc(texts[i])
          example.append(Example.from_dict(doc, annotations[i]))
        nlp.update(example,drop=0.1,losses=losses)
    print("Losses", losses)

nlp.to_disk('saved_model')

print(f'NER labels: {ner.labels}')

Pipes: ['tok2vec', 'tagger', 'parser', 'attribute_ruler']
Losses {'ner': 3.9979908708742427}


/usr/local/lib/python3.7/dist-packages/spacy/training/iob_utils.py:144: UserWarning: [W030] Some entities could not be aligned in the text "美股異動受港股收跌拖累和黃低開低走跌14%。" with entities "[(0, 2, 'FIN'), (5, 7, 'FIN'), (7, 11, 'FIN'), (11...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,


Losses {'ner': 3.550628795776848}
Losses {'ner': 2.004794759222101}
Losses {'ner': 0.39415556096606036}
Losses {'ner': 0.0020228548834966605}
Losses {'ner': 1.4039524178005655e-05}
Losses {'ner': 0.0026860001667760257}
Losses {'ner': 0.0001277679676298656}
Losses {'ner': 4.5917877661870305e-05}
Losses {'ner': 6.741614082443458e-07}
Losses {'ner': 1.1852080777490167e-05}
Losses {'ner': 2.90148512124886e-07}
Losses {'ner': 1.1559799435921993e-06}
Losses {'ner': 8.30050347437255e-05}
Losses {'ner': 1.2522446956422681e-06}
Losses {'ner': 9.506420778248617e-08}
Losses {'ner': 2.4052741616097848e-06}
Losses {'ner': 5.967667409044281e-07}
Losses {'ner': 6.734577717164194e-07}
Losses {'ner': 2.114925578442805e-07}
Losses {'ner': 1.9280684115554941e-07}
Losses {'ner': 1.3844364918176095e-08}
Losses {'ner': 3.332087169127632e-07}
Losses {'ner': 1.0613903302946832e-06}
Losses {'ner': 2.629154851320532e-07}
Losses {'ner': 2.429400193336286e-06}
Losses {'ner': 3.047125531990321e-06}
Losses {'ner': 

In [4]:
# for text, annot in TRAIN_DATA:
#     print(text,annot)
#     doc = nlp.make_doc(text)
#     biluo = spacy.training.offsets_to_biluo_tags(doc, annot["entities"])
#     print([t.text for t in doc], biluo)

In [12]:
# Let's give our trained model a try!

texts = [
  '恆指上升3.3%，是香港在這半年最大漲幅；恒生科技指數漲7.9%，創下去年7月高位。',
  '中電在假期前做好帶動，恒指單日暴升961.22點。',
  '受美股收跌拖累，匯控、和記跌超過2%。',
  '匯控下跌2%。',
  '日本、新加坡、澳洲股市單日暴升961.22點。'
]

nlp_updated = spacy.load('saved_model')

from spacy import displacy
from IPython.core.display import display, HTML

for text in texts:
  doc = nlp_updated(text)
  
  # print(doc)
  # for token in doc:
  #   print(token.text, token.pos_, token.dep_)

  # for ent in doc.ents:
    # print(ent.text, ent.start_char, ent.end_char, ent.label_)
  html = displacy.render(doc,style="ent")
  display(HTML(html))

  print('\n')

